In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [2]:
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [4]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [5]:
print(df['Geography'].head())

0    France
1     Spain
2    France
3    France
4     Spain
Name: Geography, dtype: object


In [6]:
le_gender=LabelEncoder()
df['Gender']=le_gender.fit_transform(df['Gender'])

In [7]:
ohe_geo=OneHotEncoder()
newgeo=ohe_geo.fit_transform(df[['Geography']])

In [8]:
df1=pd.DataFrame(newgeo.toarray(),columns=ohe_geo.get_feature_names_out())
df1.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [9]:
df.drop(['Geography'], axis=1,inplace=True)
df_encoded = pd.concat([df, df1], axis=1)

In [10]:
df_encoded.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
X=df_encoded.drop(['Exited'],axis=1)
y=df_encoded['Exited']
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=16,test_size=0.2)

sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [12]:
pickle.dump(le_gender,open("le_gender.pkl","wb"))
pickle.dump(ohe_geo,open("ohe_geo.pkl","wb"))
pickle.dump(sc,open("sc.pkl","wb"))

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [14]:
X_train.shape[1]

12

In [15]:
model = Sequential(
    [ 
        Dense(64, activation='relu',input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ]
)

model.summary()

c:\Users\rkdat\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,457 (13.50 KB)

 Trainable params: 3,457 (13.50 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
optad=tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optad,loss="binary_crossentropy",metrics=['accuracy'])

In [17]:
log_dir="logs/fit/"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir,histogram_freq=1)

In [18]:
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [19]:
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[tensorflow_callback,early_stopping_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8079 - loss: 0.4441 - val_accuracy: 0.8525 - val_loss: 0.3555
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8528 - loss: 0.3645 - val_accuracy: 0.8550 - val_loss: 0.3445
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8623 - loss: 0.3388 - val_accuracy: 0.8550 - val_loss: 0.3572
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8578 - loss: 0.3388 - val_accuracy: 0.8545 - val_loss: 0.3562
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8565 - loss: 0.3460 - val_accuracy: 0.8545 - val_loss: 0.3477
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8615 - loss: 0.3417 - val_accuracy: 0.8610 - val_loss: 0.3413
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8647 - loss: 0.3323 - val_accuracy: 0.8520 - val_loss: 0.3415
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8493 - loss: 0.3438 - val_accu

In [20]:
model.save('ann1.h5')

In [21]:
%load_ext tensorboard

In [22]:
%tensorboard --logdir logs/fit

In [23]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd

In [24]:
model=load_model("ann1.h5")
ohe_geo=pickle.load(open("ohe_geo.pkl","rb"))
le_gender=pickle.load(open("le_gender.pkl","rb"))

In [25]:
input_data={
    'CreditScore':800,
    'Geography':'Germany',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':75000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':60000
}

In [26]:
input_data_df=pd.DataFrame([input_data])

In [27]:
geo_encoded=ohe_geo.transform([[input_data['Geography']]])
geo_encoded_df=pd.DataFrame(geo_encoded.toarray(),columns=ohe_geo.get_feature_names_out(['Geography']))

c:\Users\rkdat\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [28]:
input_data_df=pd.concat([input_data_df.drop(['Geography'],axis=1),geo_encoded_df],axis=1)

In [29]:
input_data_df['Gender']=le_gender.transform(input_data_df['Gender'])

In [30]:
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,800,1,40,3,75000,2,1,1,60000,0.0,1.0,0.0


In [31]:
sc=pickle.load(open('sc.pkl','rb'))

In [32]:
input_data_df=sc.transform(input_data_df)
input_data_df

array([[ 1.55507074,  0.9148601 ,  0.10982325, -0.69187418, -0.01765223,
         0.7923439 ,  0.64356595,  0.96970857, -0.70342115, -1.0148604 ,
         1.74075464, -0.56888201]])

In [33]:
pred=model.predict(input_data_df)
pred_prob=pred[0][0]
if pred_prob>0.5:
    print(f'Customer is likely to Churn with probability of {pred_prob:.3f}')
else:
    print(f'Customer is not likely to Churn with probability of {pred_prob:.3f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Customer is not likely to Churn with probability of 0.067
